In [ ]:
print('Setup complete.')

# Lab 03: The ReAct Framework (Reason + Act)

## Learning Objectives
- Understand the ReAct framework for combining reasoning and acting
- Implement a loop where an LLM generates both thoughts and actions
- Use tools to gather information and update the LLM's context
- Build a simple agent that can solve a multi-step problem

## Setup

In [ ]:
import re
from typing import Dict, Any, Tuple

## Part 1: The ReAct Prompting Style

The ReAct framework instructs an LLM to follow a specific format:

1.  **Thought**: The LLM first thinks about what it needs to do to solve the problem.
2.  **Action**: Based on the thought, the LLM chooses an action to take, usually a tool to use.
3.  **Observation**: The result from the action is fed back into the prompt.
4.  **Repeat**: The LLM repeats this process until it has enough information to answer the final question.

In [ ]:
# --- Mock Tools ---
def search(query: str) -> str:
    """A mock search engine."""
    if 'capital of france' in query.lower():
        return 'The capital of France is Paris.'
    if 'population of paris' in query.lower():
        return 'The population of Paris is approximately 2.1 million.'
    return 'Information not found.'

available_tools = {
    'search': search
}

# --- Mock LLM for ReAct ---
class ReActLLM:
    def __init__(self):
        self.context_history = []

    def generate(self, prompt: str) -> str:
        self.context_history.append(prompt)
        full_context = ''.join(self.context_history)

        if 'what is the capital of france' in full_context.lower() and 'population' not in full_context.lower():
            return (
                'Thought: I need to find the capital of France. I can use the search tool for this.
'
                'Action: search[capital of France]'
            )
        elif 'capital of france is paris' in full_context.lower():
            return (
                'Thought: Now I know the capital is Paris. I need to find its population. I will use the search tool again.
'
                'Action: search[population of Paris]'
            )
        elif 'population of paris is approximately 2.1 million' in full_context.lower():
            return (
                'Thought: I have all the information I need. The capital is Paris and its population is 2.1 million. I can now provide the final answer.
'
                'Final Answer: The capital of France is Paris, which has a population of about 2.1 million.'
            )
        return 'Thought: I am unsure how to proceed. Action: finish'

## Part 2: The ReAct Agent Loop

In [ ]:
def parse_action(response: str) -> Tuple[str, str]:
    """Parses the action and its argument from the LLM's response."""
    match = re.search(r'Action: (\w+)\[(.*)\]', response)
    if match:
        return match.group(1), match.group(2) # action_name, action_input
    return None, None

def run_react_agent(question: str, llm: ReActLLM, tools: Dict, max_steps: int = 5):
    prompt = f'Question: {question}\n'
    
    for i in range(max_steps):
        print(f'--- Step {i+1} ---')
        response = llm.generate(prompt)
        print(f'LLM Output:
{response}')
        
        if 'Final Answer:' in response:
            final_answer = response.split('Final Answer:')[-1].strip()
            print(f'\nFinal Answer Found: {final_answer}')
            return
            
        action_name, action_input = parse_action(response)
        if action_name and action_name in tools:
            tool_function = tools[action_name]
            observation = tool_function(action_input)
            print(f'Observation: {observation}')
            prompt = f'\nObservation: {observation}\n' # Next prompt is just the new info
        else:
            print('No valid action found or action finished.')
            break

# --- Run the Agent ---
question = 'What is the capital of France and what is its population?'
react_llm = ReActLLM()

run_react_agent(question, react_llm, available_tools)

## Exercises

1. **Add a New Tool**: Create a new tool called `get_current_date()` that returns today's date as a string. Add it to the `available_tools` dictionary. Then, modify the `ReActLLM` to handle a question like, "What is the date today?".
2. **Handle Tool Errors**: Modify the `run_react_agent` loop. What if a tool fails and raises an exception? The loop should catch the error and feed an observation back to the LLM like `Observation: Error executing tool: [error message]`.
3. **Improve the Prompt**: The initial prompt is very simple. A better prompt would include a description of the available tools and a few examples of the Thought-Action-Observation format (a few-shot prompt). Write a more detailed initial prompt for the agent.

## Summary

You learned:
- The **ReAct framework**, which synergistically combines reasoning (Thought) and tool use (Action) in an iterative loop.
- How to create an **agent loop** that parses LLM output, executes tools, and feeds the results back to the model.
- That by breaking down a complex question into smaller, tool-assisted steps, an LLM can solve problems it couldn't answer in a single pass.